LAB 1 SSN MICHAŁ SIENKIEWICZ

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Definicja pierwszej warstwy konwolucyjnej: wejście 3 kanały (RGB), wyjście 32 kanały, rozmiar filtra 3x3, padding=1
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        # Definicja drugiej warstwy konwolucyjnej: wejście 32 kanały, wyjście 64 kanały, rozmiar filtra 3x3, padding=1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Definicja trzeciej warstwy konwolucyjnej: wejście 64 kanały, wyjście 128 kanały, rozmiar filtra 3x3, padding=1
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # Warstwa dropout1 z prawdopodobieństwem wyzerowania 25% neuronów
        self.dropout1 = nn.Dropout(0.25)
        # Warstwa dropout2 z prawdopodobieństwem wyzerowania 50% neuronów
        self.dropout2 = nn.Dropout(0.5)
        # Warstwa w pełni połączona (fully connected), wejście 128 * 4 * 4 neurony (wynik rozmiaru tensora po ostatniej konwolucji), wyjście 512 neurony
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        # Warstwa w pełni połączona (fully connected), wejście 512 neurony, wyjście 10 neurony (liczba klas)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        # Przekształcenie wejścia przez pierwszą warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv1(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Przekształcenie przez drugą warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv2(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Przekształcenie przez trzecią warstwę konwolucyjną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.conv3(x))
        # Redukcja wymiarowości przez operację max-pooling z rozmiarem okna 2x2
        x = nn.functional.max_pool2d(x, 2)
        # Wykorzystanie warstwy dropout1
        x = self.dropout1(x)
        # Spłaszczenie tensora do postaci wektora przed podaniem na warstwę w pełni połączoną
        x = torch.flatten(x, 1)
        # Przekształcenie przez warstwę w pełni połączoną, zastosowanie funkcji aktywacji ReLU
        x = nn.functional.relu(self.fc1(x))
        # Wykorzystanie warstwy dropout2
        x = self.dropout2(x)
        # Przekształcenie przez warstwę w pełni połączoną (wyjściową)
        x = self.fc2(x)
        # Zastosowanie funkcji log_softmax dla uzyskania prawdopodobieństw przynależności do klas
        output = nn.functional.log_softmax(x, dim=1)
        return output

In [4]:
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [5]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

[1, 100] loss: 2.271
[1, 200] loss: 2.090
[1, 300] loss: 1.950
[1, 400] loss: 1.862
[1, 500] loss: 1.759
[1, 600] loss: 1.720
[1, 700] loss: 1.641
[2, 100] loss: 1.587
[2, 200] loss: 1.547
[2, 300] loss: 1.531
[2, 400] loss: 1.459
[2, 500] loss: 1.441
[2, 600] loss: 1.442
[2, 700] loss: 1.413
[3, 100] loss: 1.324
[3, 200] loss: 1.358
[3, 300] loss: 1.317
[3, 400] loss: 1.276
[3, 500] loss: 1.244
[3, 600] loss: 1.214
[3, 700] loss: 1.223
[4, 100] loss: 1.180
[4, 200] loss: 1.171
[4, 300] loss: 1.121
[4, 400] loss: 1.119
[4, 500] loss: 1.108
[4, 600] loss: 1.116
[4, 700] loss: 1.104
[5, 100] loss: 1.058
[5, 200] loss: 1.046
[5, 300] loss: 1.050
[5, 400] loss: 1.043
[5, 500] loss: 1.017
[5, 600] loss: 1.009
[5, 700] loss: 0.998
[6, 100] loss: 0.975
[6, 200] loss: 0.954
[6, 300] loss: 0.963
[6, 400] loss: 0.974
[6, 500] loss: 0.955
[6, 600] loss: 0.949
[6, 700] loss: 0.950
[7, 100] loss: 0.927
[7, 200] loss: 0.924
[7, 300] loss: 0.923
[7, 400] loss: 0.916
[7, 500] loss: 0.916
[7, 600] loss

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

Accuracy on the test set: 74.56%


ZAD.1.

MNIST

In [12]:
# przygotowanie transformacji dla zestawu danych MNIST (liczby ręcznie pisane)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(((0.5,)),  (0.5,))
])

In [13]:
# pobranie zestawu danych MNIST
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [15]:
#model sieci neuronowej dla MNIST
class NetMNIST(nn.Module):
    def __init__(self):
        super(NetMNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)

        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

In [16]:
model = NetMNIST()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [17]:

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

[1, 100] loss: 1.440
[1, 200] loss: 0.428
[1, 300] loss: 0.265
[1, 400] loss: 0.198
[1, 500] loss: 0.168
[1, 600] loss: 0.161
[1, 700] loss: 0.140
[1, 800] loss: 0.138
[1, 900] loss: 0.126
[2, 100] loss: 0.105
[2, 200] loss: 0.102
[2, 300] loss: 0.114
[2, 400] loss: 0.110
[2, 500] loss: 0.097
[2, 600] loss: 0.105
[2, 700] loss: 0.087
[2, 800] loss: 0.093
[2, 900] loss: 0.090
[3, 100] loss: 0.079
[3, 200] loss: 0.088
[3, 300] loss: 0.084
[3, 400] loss: 0.077
[3, 500] loss: 0.080
[3, 600] loss: 0.072
[3, 700] loss: 0.073
[3, 800] loss: 0.075
[3, 900] loss: 0.088
[4, 100] loss: 0.084
[4, 200] loss: 0.070
[4, 300] loss: 0.072
[4, 400] loss: 0.072
[4, 500] loss: 0.064
[4, 600] loss: 0.067
[4, 700] loss: 0.064
[4, 800] loss: 0.064
[4, 900] loss: 0.062
[5, 100] loss: 0.054
[5, 200] loss: 0.054
[5, 300] loss: 0.060
[5, 400] loss: 0.067
[5, 500] loss: 0.066
[5, 600] loss: 0.059
[5, 700] loss: 0.060
[5, 800] loss: 0.060
[5, 900] loss: 0.058
[6, 100] loss: 0.053
[6, 200] loss: 0.058
[6, 300] loss

In [19]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on the test set: {100 * correct / total:.2f}%")

Accuracy on the test set: 98.27%


In [ ]:
# draw loss function for MNIST pytorch model 
print()